<a href="https://colab.research.google.com/github/2stacks/QuantizeLLMs/blob/main/QuantizeLLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!!git clone https://github.com/ggerganov/llama.cpp

In [ ]:
!cd llama.cpp && GGML_CUDA=1 cmake -B build -DGGML_CUDA=ON && cmake --build build --config Release

In [ ]:
!cd llama.cpp && pip install -r requirements.txt

In [ ]:
from huggingface_hub import snapshot_download

In [ ]:
model_name = "google/gemma-2b-it"

In [ ]:
methods = ["q4_k_m"]

In [ ]:
base_model = "./orignal_model/"

In [ ]:
quantized_path = "./quantized_model/"

In [ ]:
snapshot_download(repo_id=model_name, local_dir=base_model)

In [ ]:
orignal_model = quantized_path+"/fp16.gguf"

In [ ]:
!mkdir ./quantized_model/

In [ ]:
!python llama.cpp/convert_hf_to_gguf.py ./orignal_model/ --outtype f16 --outfile ./quantized_model/FP16.gguf

In [ ]:
import os

In [ ]:
for m in methods:
  qtype = f"{quantized_path}/{m.upper()}.gguf"
  os.system("./llama.cpp/build/bin/llama-quantize "+quantized_path+"/FP16.gguf "+qtype+" "+m)

In [ ]:
! ./llama.cpp/build/bin/llama-cli -m ./quantized_model/Q4_K_M.gguf -n 90 --repeat_penalty 1.0 --color -i -r "User: " -f llama.cpp/prompts/chat-with-bob.txt

In [ ]:
from huggingface_hub import HfApi, HfFolder, create_repo, upload_file

In [ ]:
model_path = "./quantized_model/Q4_K_M.gguf"
repo_name = "gemma-2b-it-GGUF-quantized"
repo_url = create_repo(repo_name, private=False)

In [ ]:
api = HfApi()

In [ ]:
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="Q4_K_M.gguf",
    repo_id= "2stacks/gemma-2b-it-GGUF-quantized",
    repo_type="model",
)